# Multi-Agent System Demo
## Economic Forecasting Team in Action - Notebook 4

**Objective**: Demonstrate the complete multi-agent system where specialized agents collaborate as a team for economic forecasting.

### What You'll Learn:
- Full multi-agent workflow orchestration
- Agent coordination and communication
- Session management and state persistence
- Real-time collaboration between specialized agents
- Team coordinator agent functionality

## 1. Setup and Team Initialization

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import asyncio
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

# Import the complete multi-agent system
from agents.team_coordinator import EconomicTeamCoordinator
from google.adk.models.google_llm import Gemini
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

# Setup visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Multi-agent system components imported successfully")

In [ ]:
# Initialize the complete economic forecasting team
print("🚀 Initializing Economic Forecasting Team...")

# Configuration
bea_api_key = os.getenv('BEA_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if not bea_api_key or not google_api_key:
    print("❌ Please set BEA_API_KEY and GOOGLE_API_KEY in your .env file")
else:
    print("✅ API keys loaded successfully")

# Configure retry options
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Initialize model
model = Gemini(
    model="gemini-2.0-flash-exp",
    retry_options=retry_config
)

# Create the complete team coordinator
team_coordinator = EconomicTeamCoordinator(bea_api_key, model)

print("🤖 Economic Forecasting Team Initialized!")
print("\n👥 Team Members:")
print("   • Data Collector Agent - Fetches economic data from BEA API")
print("   • Economic Analyst Agent - Analyzes trends and indicators")
print("   • Forecasting Specialist Agent - Builds models and predictions")
print("   • Visualization Agent - Creates dashboards and reports")
print("   • Team Coordinator - Orchestrates the entire workflow")

## 2. Team Capabilities Demonstration

In [ ]:
# Test individual agent capabilities
async def test_individual_agents():
    """Test each agent's individual capabilities"""

    print("🧪 Testing Individual Agent Capabilities...")

    # Test Data Collector Agent
    print("\n📥 Data Collector Agent Test:")
    gdp_result = await team_coordinator.data_collector.get_gdp_data()
    if gdp_result['status'] == 'success':
        print(f"   ✅ GDP data collected: {len(gdp_result['data'])} points")
    else:
        print(f"   ❌ GDP data collection failed")

    # Test Economic Analyst Agent
    print("\n📊 Economic Analyst Agent Test:")
    if gdp_result['status'] == 'success' and gdp_result['data']:
        analysis_result = await team_coordinator.economic_analyst.analyze_growth_trends(gdp_result['data'])
        if analysis_result['status'] == 'success':
            print(f"   ✅ Growth analysis completed: {analysis_result.get('trend', 'N/A')} trend")
        else:
            print(f"   ❌ Growth analysis failed")

    # Test Forecasting Specialist Agent
    print("\n🔮 Forecasting Specialist Agent Test:")
    if gdp_result['status'] == 'success' and gdp_result['data']:
        forecast_result = await team_coordinator.forecasting_specialist.forecast_gdp(gdp_result['data'], horizon=4)
        if forecast_result['status'] == 'success':
            print(f"   ✅ GDP forecast generated: {forecast_result.get('horizon', 'N/A')} periods")
        else:
            print(f"   ❌ GDP forecasting failed")

    # Test Visualization Agent
    print("\n📈 Visualization Agent Test:")
    if gdp_result['status'] == 'success' and gdp_result['data']:
        viz_result = await team_coordinator.visualization_agent.create_growth_chart(gdp_result['data'])
        if viz_result['status'] == 'success':
            print(f"   ✅ Growth chart created successfully")
        else:
            print(f"   ❌ Visualization creation failed")

# Run individual agent tests
await test_individual_agents()

## 3. Complete Workflow Demonstration

In [ ]:
# Run complete economic analysis workflow
print("🔄 Running Complete Economic Analysis Workflow...")

workflow_query = """
Please coordinate the economic forecasting team to perform a comprehensive analysis:

1. DATA COLLECTION:
   - Collect the latest GDP data from BEA
   - Gather recent unemployment statistics
   - Retrieve current inflation indicators

2. ECONOMIC ANALYSIS:
   - Analyze current GDP growth trends
   - Calculate key economic indicators
   - Identify business cycle position
   - Detect any economic anomalies

3. FORECASTING:
   - Generate 8-quarter GDP forecasts
   - Provide confidence intervals
   - Create ensemble predictions

4. VISUALIZATION & REPORTING:
   - Create comprehensive economic dashboard
   - Generate forecast visualizations
   - Prepare executive summary

Please ensure all agents collaborate effectively and provide actionable insights.
"""

print("📝 Workflow Query:")
print(workflow_query)
print("\n" + "="*60)

In [ ]:
# Execute the complete workflow
print("🚀 Executing Multi-Agent Workflow...")

workflow_results = await team_coordinator.run_complete_analysis(workflow_query)

if workflow_results['status'] == 'success':
    print("✅ Multi-Agent Workflow Completed Successfully!")
    print(f"📋 Session ID: {workflow_results['session_id']}")

    # Display the team's final response
    if 'final_response' in workflow_results['results']:
        print("\n🤖 TEAM COORDINATOR RESPONSE:")
        print("=" * 50)

for part in workflow_results['results']['final_response'].parts:
if hasattr(part, 'text') and part.text:
print(part.text)
else:
    print("❌ Multi-Agent Workflow Failed")
    print(f"Error: {workflow_results.get('error_message', 'Unknown error')}")

## 4. Session Management and State Persistence

In [ ]:
# Demonstrate session management capabilities
print("💾 Session Management Demonstration")
print("=" * 50)

if workflow_results['status'] == 'success':
    # Retrieve the session to examine stored data
    session = await team_coordinator.session_service.get_session(
        app_name="economic_forecasting",
        user_id="user",
        session_id=workflow_results['session_id']
    )

    print(f"📊 Session Analysis:")
    print(f"   Session ID: {session.id}")
    print(f"   Created: {session.created_time}")
    print(f"   Events: {len(session.events)} interactions")
    print(f"   State variables: {len(session.state)}")

    # Show session events
    print(f"\n📋 Recent Session Events:")
    for i, event in enumerate(session.events[-5:]):  # Show last 5 events
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text:
                    preview = part.text[:100] + "..." if len(part.text) > 100 else part.text
                    print(f"   {i+1}. {event.author}: {preview}")
                    break
else:
    print("❌ No session available for analysis")

In [ ]:
# Demonstrate session continuation
print("\n🔄 Session Continuation Demonstration")
print("=" * 50)

if workflow_results['status'] == 'success':
    follow_up_query = """
    Based on your previous analysis, please provide:
    1. What are the key risks to the economic outlook?
    2. Which sectors show the most promise for growth?
    3. What policy recommendations would you suggest?
    """

    print("📝 Follow-up Query:")
    print(follow_up_query)

    # Continue the same session
    print("\n🔄 Continuing existing session...")

    follow_up_results = await team_coordinator.run_complete_analysis(follow_up_query)

    if follow_up_results['status'] == 'success':
        print("✅ Follow-up Analysis Completed!")
        print(f"📋 Same Session ID: {follow_up_results['session_id']}")

        # Show the team's continued response
        if 'final_response' in follow_up_results['results']:
            print("\n🤖 TEAM COORDINATOR FOLLOW-UP:")
            print("=" * 50)

for part in follow_up_results['results']['final_response'].parts:
if hasattr(part, 'text') and part.text:
print(part.text)
    else:
        print("❌ Follow-up analysis failed")
else:
    print("❌ No existing session to continue")

## 5. Real-time Collaboration Visualization

In [ ]:
# Create visualization of agent collaboration
print("👥 Agent Collaboration Visualization")
print("=" * 50)

# Create a flowchart of the multi-agent workflow
fig = go.Figure()

# Define agent positions and connections
agents = {
    'User': {'x': 0, 'y': 0, 'color': '#1f77b4'},
    'Team Coordinator': {'x': 1, 'y': 0, 'color': '#ff7f0e'},
    'Data Collector': {'x': 2, 'y': 1, 'color': '#2ca02c'},
    'Economic Analyst': {'x': 2, 'y': 0, 'color': '#d62728'},
    'Forecasting Specialist': {'x': 2, 'y': -1, 'color': '#9467bd'},
    'Visualization Agent': {'x': 3, 'y': 0, 'color': '#8c564b'},
    'Final Report': {'x': 4, 'y': 0, 'color': '#e377c2'}
}

# Add agent nodes
for agent, props in agents.items():
    fig.add_trace(go.Scatter(
        x=[props['x']], y=[props['y']],
        mode='markers+text',
        marker=dict(size=40, color=props['color'], line=dict(width=2, color='DarkSlateGrey')),
        text=agent,
        textposition="middle center",
        name=agent,
        hoverinfo='text'
    ))

# Define workflow connections
connections = [
    ('User', 'Team Coordinator'),
    ('Team Coordinator', 'Data Collector'),
    ('Team Coordinator', 'Economic Analyst'),
    ('Team Coordinator', 'Forecasting Specialist'),
    ('Data Collector', 'Economic Analyst'),
    ('Data Collector', 'Forecasting Specialist'),
    ('Economic Analyst', 'Visualization Agent'),
    ('Forecasting Specialist', 'Visualization Agent'),
    ('Visualization Agent', 'Final Report'),
    ('Final Report', 'User')
]

# Add connection lines
for start, end in connections:
    fig.add_trace(go.Scatter(
        x=[agents[start]['x'], agents[end]['x']],
        y=[agents[start]['y'], agents[end]['y']],
        mode='lines',
        line=dict(color='gray', width=2, dash='dot'),
        showlegend=False
    ))

fig.update_layout(
    title='Multi-Agent Economic Forecasting Team Workflow',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    showlegend=False,
    height=500,
    plot_bgcolor='white'
)

fig.show()

## 6. Advanced Multi-Agent Scenarios

In [ ]:
# Demonstrate complex multi-agent scenarios
print("🎯 Advanced Multi-Agent Scenarios")
print("=" * 50)

async def run_complex_scenario():
    """Run a complex scenario requiring deep agent collaboration"""

    complex_query = """
    We're facing potential economic uncertainty. Please coordinate the team to:

    SCENARIO ANALYSIS:
    1. Collect data on recent economic shocks or anomalies
    2. Analyze the impact on different economic sectors
3. Assess vulnerability in employment and inflation
    4. Model different recovery scenarios

    STRESS TESTING:
    1. Run pessimistic, baseline, and optimistic forecasts
    2. Test economic resilience under different conditions
    3. Identify early warning indicators

    STRATEGIC RECOMMENDATIONS:
    1. Recommend monitoring priorities
    2. Suggest policy interventions if needed
    3. Provide contingency planning guidance

    Please have all agents work together to provide a comprehensive assessment.
    """

    print("📝 Complex Scenario Query:")
    print(complex_query)

    print("\n🚀 Executing Complex Scenario Analysis...")

    scenario_results = await team_coordinator.run_complete_analysis(complex_query)

    if scenario_results['status'] == 'success':
        print("✅ Complex Scenario Analysis Completed!")
        print(f"📋 Session ID: {scenario_results['session_id']}")

        # Show key insights
        if 'final_response' in scenario_results['results']:
            print("\n🤖 TEAM COORDINATOR SCENARIO ANALYSIS:")
            print("=" * 50)

for part in scenario_results['results']['final_response'].parts:
if hasattr(part, 'text') and part.text:
print(part.text)
    else:
        print("❌ Complex scenario analysis failed")

# Run the complex scenario
await run_complex_scenario()

## 7. Performance Monitoring

In [ ]:
# Monitor multi-agent system performance
print("📊 Multi-Agent System Performance")
print("=" * 50)

def analyze_agent_performance():
    """Analyze and display agent performance metrics"""

    # Simulated performance metrics (in production, these would be real metrics)
    performance_data = {
        'Agent': ['Data Collector', 'Economic Analyst', 'Forecasting Specialist', 'Visualization Agent', 'Team Coordinator'],
        'Success Rate (%)': [98.5, 95.2, 92.8, 96.7, 94.3],
        'Avg Response Time (s)': [2.1, 3.8, 7.2, 4.5, 12.3],
        'Tools Used': [3, 4, 4, 4, 5],
        'Collaboration Score': [85, 90, 88, 87, 95]
    }

    perf_df = pd.DataFrame(performance_data)

    print("📈 Agent Performance Metrics:")
    print(perf_df.to_string(index=False))

    # Create performance visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Success Rate', 'Response Time', 'Tools Used', 'Collaboration Score'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}], [{'type': 'bar'}, {'type': 'bar'}]]
    )

    # Success Rate
    fig.add_trace(
        go.Bar(x=perf_df['Agent'], y=perf_df['Success Rate (%)'], name='Success Rate'),
        row=1, col=1
    )

    # Response Time
    fig.add_trace(
        go.Bar(x=perf_df['Agent'], y=perf_df['Avg Response Time (s)'], name='Response Time'),
        row=1, col=2
    )

    # Tools Used
    fig.add_trace(
        go.Bar(x=perf_df['Agent'], y=perf_df['Tools Used'], name='Tools Used'),
        row=2, col=1
    )

    # Collaboration Score
    fig.add_trace(
        go.Bar(x=perf_df['Agent'], y=perf_df['Collaboration Score'], name='Collaboration'),
        row=2, col=2
    )

    fig.update_layout(
        title_text="Multi-Agent System Performance Dashboard",
        height=600,
        showlegend=False
    )

    fig.show()

# Display performance analysis
analyze_agent_performance()

## 8. Summary and Production Insights

In [ ]:
print("🎯 NOTEBOOK 4 SUMMARY")
print("=" * 50)

print("✅ Multi-Agent System Successfully Demonstrated!")
print("\n👥 Team Coordination Achieved:")
print("   • Data Collector → Economic Analyst → Forecasting Specialist → Visualization Agent")
print("   • Seamless handoffs between specialized agents")
print("   • Coordinated workflow managed by Team Coordinator")

print("\n💾 Session Management Features:")
print("   • Persistent conversation state")
print("   • Context maintenance across interactions")
print("   • Resumable workflows")

print("\n🔧 Advanced Capabilities Demonstrated:")
print("   • Complex scenario analysis")
print("   • Real-time agent collaboration")
print("   • Performance monitoring")
print("   • Workflow visualization")

print("\n🔜 Next Steps:")
print("   1. Proceed to Notebook 5: Interactive Economic Dashboard")
print("   2. Explore Notebook 6: Agent Evaluation and Testing")
print("   3. Review Notebook 7: Production Deployment Guide")

print("\n💡 Production Insights:")
print("   • Multi-agent systems excel at complex, multi-step tasks")
print("   • Specialized agents provide domain expertise")
print("   • Team coordination ensures comprehensive analysis")
print("   • Session management enables ongoing conversations")
print("   • Performance monitoring is crucial for reliability")

print("\n🚀 Ready for Production Deployment!")
print("   The multi-agent system is now fully functional and ready for real-world economic forecasting tasks.")